Objetivo:
API -> DYNAMO

API -> S3

Consultar as APIs e gravar os dados nos recursos s3 e DynamoDB da AWS. Via API (utilizando Python).


In [ ]:
%%sh
sudo pip install s3fs
sudo pip install boto3

  Created wheel for aiobotocore: filename=aiobotocore-2.2.0-py3-none-any.whl size=57108 sha256=53ac2c730f348c826f7f4be0a1d99901afd3be21c892892fac166d3624a52708
  Stored in directory: /root/.cache/pip/wheels/5f/c1/90/d81f357ee479228ab98b7558b6a6af12234e5ee9c29ab5c16f
Successfully built aiobotocore
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.21
    Uninstalling botocore-1.24.21:
      Successfully uninstalled botocore-1.24.21


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.2.0 requires botocore<1.24.22,>=1.24.21, but you have botocore 1.25.2 which is incompatible.


In [ ]:
#import das bibliotecas
import json,s3fs,boto3,requests as r
from decimal import Decimal

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
#Chaves de acesso AWS
aws_access_key_id = ''
aws_secret_access_key = ''
aws_session_token = ''

In [ ]:
#Client s3
client_s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key,
                      aws_session_token=aws_session_token, use_ssl=False)
#s3fs
sf = s3fs.S3FileSystem(key =aws_access_key_id,
                       secret=aws_secret_access_key,
                       token =aws_session_token)

In [ ]:
#Criar bucket
bucket_space = 'spacex-2102100'
client_s3.create_bucket(Bucket=bucket_space)

{'Location': '/spacex-2102100',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Thu, 28 Apr 2022 00:05:55 GMT',
   'location': '/spacex-2102100',
   'server': 'AmazonS3',
   'x-amz-id-2': 'is22LevjdKyN1IurGCjzT4Pxn8NTLmsxPaOhN7fYXC5+oH/8So8Jk6pecr3RHfay9w7SD9RtC28=',
   'x-amz-request-id': '53ABZFKYMD6EG7PD'},
  'HTTPStatusCode': 200,
  'HostId': 'is22LevjdKyN1IurGCjzT4Pxn8NTLmsxPaOhN7fYXC5+oH/8So8Jk6pecr3RHfay9w7SD9RtC28=',
  'RequestId': '53ABZFKYMD6EG7PD',
  'RetryAttempts': 0}}

In [ ]:
#Resource dynamo
dynamodb = boto3.resource('dynamodb', aws_access_key_id=aws_access_key_id,
                      aws_secret_access_key=aws_secret_access_key,
                      aws_session_token=aws_session_token,
                      region_name = "us-east-1", use_ssl=False)

In [ ]:
#Criação da tabela capsule
capsule = 'capsule' #nome da tabela
params = {
        'TableName': capsule, 
        #Esquema das chaves
        'KeySchema': [
            {'AttributeName': 'capsule_serial', 'KeyType': 'HASH'}, # hash será a chave primaria
            {'AttributeName': 'capsule_id', 'KeyType': 'RANGE'} #range, não obrigatória, formaria uma chave composta
        ],
        #Tipo de dados da chave
        'AttributeDefinitions': [ 
            {'AttributeName': 'capsule_serial', 'AttributeType': 'S'}, #S de string
            {'AttributeName': 'capsule_id', 'AttributeType': 'S'} #N de Number
        ],
        #Configuração de escrita e leitura por segundo
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    }
dynamodb.create_table(**params)

dynamodb.Table(name='capsule')

In [ ]:
#Criação da tabela rocket
rocket = 'rocket' #nome da tabela
params = {
        'TableName': rocket, 
        #Esquema das chaves
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}, # hash será a chave primaria 
        ],
        #Tipo de dados da chave
        'AttributeDefinitions': [ 
            {'AttributeName': 'id', 'AttributeType': 'N'}, #S de string
        ],
        #Configuração de escrita e leitura por segundo
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    }
dynamodb.create_table(**params)

dynamodb.Table(name='rocket')

In [ ]:
#Criação da tabela dragon
dragon = 'dragon' #nome da tabela
params = {
        'TableName': dragon, 
        #Esquema das chaves
        'KeySchema': [
            {'AttributeName': 'id', 'KeyType': 'HASH'}, # hash será a chave primaria
            {'AttributeName': 'name', 'KeyType': 'RANGE'} #range, não obrigatória, formaria uma chave composta
        ],
        #Tipo de dados da chave
        'AttributeDefinitions': [ 
            {'AttributeName': 'id', 'AttributeType': 'S'}, #S de string
            {'AttributeName': 'name', 'AttributeType': 'S'} #N de Number
        ],
        #Configuração de escrita e leitura por segundo
        'ProvisionedThroughput': {
            'ReadCapacityUnits': 3,
            'WriteCapacityUnits': 3
        }
    }
dynamodb.create_table(**params)

dynamodb.Table(name='dragon')

## Usando a API Capsules

In [ ]:
#Buscar tabela capsule no Dynamo
capsule_dynamo = dynamodb.Table('capsule')

In [ ]:
#Busca dados na api através da lib requests.
url_capsules = 'https://api.spacexdata.com/v3/capsules'
dados = r.get(url_capsules)
j_capsules = json.loads(dados.text,parse_float=Decimal)

In [ ]:
#Grava dados no bucket do s3 - informar o nome do bucket e o caminho 
diretorio_capsules = 'capsules/capsules.json' #diretorio/nome_arquivo.json
caminho_capsules = bucket_space + '/' + diretorio_capsules
with sf.open(caminho_capsules,'w') as f:
  #dump é o comando para gravar o dado
    json.dump(j_capsules,f)
    from decimal import Decimal
    json.loads(dados.text,parse_float=Decimal)

In [ ]:
#Insere dados no dynamodb
#Faz um "for" no json e vai inserindo registro a registro
for i in j_capsules:
  capsule_dynamo.put_item(
      Item=i
      )

## Usando a API Rocket

In [ ]:
#Buscar tabela rocket no Dynamo
rocket_dynamo = dynamodb.Table('rocket')

In [ ]:
#Busca dados na api através da lib requests.
from decimal import Decimal
url_rocket = 'https://api.spacexdata.com/v3/rockets'
dados = r.get(url_rocket)
j_rocket = json.loads(dados.text,parse_float=Decimal)

In [ ]:
#Grava dados no bucket do s3 - informar o nome do bucket e o caminho 
diretorio_rocket = 'rockets/rockets.json' #diretorio/nome_arquivo.json
caminho_rocket = bucket_space + '/' + diretorio_rocket
j = json.loads(dados.text)
with sf.open(caminho_rocket,'w') as f:
  json.dump(j,f)
  
#dump é o comando para gravar o dado

In [ ]:
#Insere dados no dynamodb
#Faz um "for" no json e vai inserindo registro a registro
url_rocket = 'https://api.spacexdata.com/v3/rockets'
dados = r.get(url_rocket)
j_rocket = json.loads(dados.text,parse_float=Decimal)
for i in j_rocket:
  rocket_dynamo.put_item(
      Item=i
)

##USANDO a API Dragon

In [ ]:
#Buscar tabela rocket no Dynamo
dragon_dynamo = dynamodb.Table('dragon')

In [ ]:
#Busca dados na api através da lib requests.
url_dragon = 'https://api.spacexdata.com/v3/dragons'
dados = r.get(url_dragon)
j_dragon = json.loads(dados.text,parse_float=Decimal)

In [ ]:
#Grava dados no bucket do s3 - informar o nome do bucket e o caminho 
diretorio_dragon = 'dragons/dragons.json' #diretorio/nome_arquivo.json
caminho_dragon = bucket_space + '/' + diretorio_dragon
j = json.loads(dados.text)
with sf.open(caminho_dragon,'w') as f:
  json.dump(j,f)
  

In [ ]:
url_dragon = 'https://api.spacexdata.com/v3/dragons'
dados = r.get(url_dragon)
j_dragon = json.loads(dados.text,parse_float=Decimal)
for i in j_dragon:
  dragon_dynamo.put_item(
      Item=i
)